In [59]:
import openpyxl
import os, sys
import glob
import psycopg2
import json
from arcgis.gis import GIS, Item
import fiona
from fiona.crs import from_epsg
import zipfile


##### Main Function
def main():
    
    # Connect to local db
    conn_string = "host='10.94.12.55' port=5432 dbname='dotdb' user='postgres' password='Feb241989'"
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    
    # Specify the folder path and extension of LAPD update worksheet, retrieve rows
    lapd_fldr_path = "Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/"
    extension = "xlsx"
    
    # Access first workbook with extension
    os.chdir(lapd_fldr_path)
    result = [i for i in glob.glob('*.{}'.format(extension))]
    lapd_file = lapd_fldr_path + result[0]

    # Load active sheet
    lapd_wb = openpyxl.load_workbook(lapd_file, read_only=True)
    lapd_ws = lapd_wb.active
    rows = lapd_ws.rows
    first_row = [cell.value for cell in next(rows)]
    
    # Create the dictionary 
    final_rows = CollisionDict(rows, first_row)

    # Only commit if the spreadsheet imported correctly
    if len(final_rows) > 0:

        # Delete old data from db table 'geom_lapd_collisions'
        cursor.execute("""DELETE FROM geom_lapd_collisions_xl""")
        conn.commit()
        

        # Load new data into db table 'geom_lapd_collisions'
        for row in final_rows:
            
            # Assign collisons without a location to X,Y coordinates (0,0)
            if row['XY'] == "":
                row['XY'] = 'POINT(0 0)'
            else:
                row['XY'] = 'POINT(' + row['XY'].replace(","," ") + ')'
            
            # Reformat time value
            row['BEGTIME'] = str(row['BEGTIME']).zfill(4)
            
            
            #print(row['DR'],row['XY'])
            
            # Format and insert row into db
            cursor.execute("INSERT INTO geom_lapd_collisions_xl (dr_no, status, crm_cd, rd, day_of_week, date_occ, time_occ,"
                           "location, collision_severity, ped_inv, bike_inv, mc_inv, hit_and_run,"
                           "at_intersection, narrative, vic_sex, vic_age, vic_dob, coord_x," 
                           "coord_y, coord_xy, matched, score, loc_tier, geom)" 
                           "VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,ST_Transform(ST_GeomFromText(%s,3857),4326));",  
                          (
                          row['DR'], #dr_no
                          row['CASESTATUS'], #status
                          row['CRIMECLASSCODES'], #crm_cd
                          row['RD'], #rd
                          row['DOW'], #day_of_week
                          row['BEGDATE'].date(), #date_occ
                          str(row['BEGTIME']).zfill(4), #time_occ
                          row['STREETNAME1'], #location #replace 'CRIMELOC' with STREETNAME1
                          row['collision_severity'], #collision_severity
                          row['ped_inv'], #ped_inv
                          row['bike_inv'], #bike_inv
                          row['mc_inv'], #mc_inv
                          row['hit_and_run'], #hit_and_run
                          row['intersection'], #at_intersection
                          row['NARRATIVE'], #narrative
                          row['VICSEX'], #vic_sex
                          row['VICAGE'], #vic_age
                          row['VICDOB'], #vic_dob
                          row['X'], #coord_x
                          row['Y'], #coord_y
                          row['XY'], #coord_xy
                          row['MATCHED'], #matched
                          row['SCORE'], #score
                          row['LOC_TIER'], #loc_tier
                          row['XY'] #geom
                          ))
    
    else:
        print("No Rows Found")
    
    # Commit changes
    conn.commit()
    print("committed")
    
    ##### Export updated data as a shapefile
    # Build feature collection from current rows in geom_lapd_collisions_xl
    build_feature_collection_query = """
        SELECT jsonb_build_object(
            'type',     'FeatureCollection',
            'features', jsonb_agg(feature)
        )
        FROM (
          SELECT jsonb_build_object(
            'type',       'Feature',
            'id',         collision_pkey,
            'geometry',   ST_AsGeoJSON(geom)::jsonb,
            'properties', to_jsonb(row) - 'collision_pkey' - 'geom'
          ) AS feature
          FROM (SELECT * FROM geom_lapd_collisions_xl) row) features;"""  
    cursor.execute(build_feature_collection_query)
    fc = cursor.fetchall()
    
    # Grab the SRID of the geometry column from PostGIS table
    srid_query = """SELECT Find_SRID('public', 'geom_lapd_collisions_xl', 'geom');"""
    cursor.execute(srid_query)
    geom_srid = cursor.fetchone()[0]
    print(geom_srid)
    geom_srid = 3857
    
    # Schema for Fiona output
    out_schema = {
        'geometry': 'Point',
        'properties': {
            'rd':'str',
            'dr_no':'str',
            'score':'int',
            'crm_cd':'int',
            'mc_inv':'str',
            'status':'str',
            'coord_x':'float',
            'coord_y':'float',
            'matched':'str',
            'ped_inv':'str',
            'vic_age':'str',
            'vic_dob':'str',
            'vic_sex':'str',
            'bike_inv':'str',
            'coord_xy':'str',
            'date_occ':'str',
            'loc_tier':'str',
            'location':'str',
            'time_occ':'str',
            'narrative':'str',
            'day_of_week':'str',
            'hit_and_run':'str',
            'at_intersection':'str',
            'collision_severity':'str'
        }
    }
    
    # Using Fiona, output feature collection to Shapefile format
    out_dir = 'Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp/'
    out_file = 'collisions.shp'
    out_path = out_dir + out_file
    with fiona.open(out_path,'w', crs=from_epsg(geom_srid), driver='ESRI Shapefile', schema=out_schema) as c:
        for record in fc[0][0]['features']:
            c.write(record)
    print("written shapefile to disk.")
    
    # Close the cursor and the connection
    conn.close()
    
    ##### Zip the recently created shapefile directory
    # Get a list of files in the shapefile directory, run makeArchive function
    fileList = glob.glob(out_dir + '*')
    shp_zip = 'Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/Collisions.zip'
    makeArchive(fileList, shp_zip, out_dir)
    
    ##### Login to AGOL and Update Item 
    # Login
    username = "timothy.black_ladot"
    password = "Feb241989"
    gis = GIS(url="https://www.arcgis.com", username=username, password=password)
    
    # Update item in AGOL
    shp_path = shp_zip
    shp_properties = {
        'title':'Collisions 2017 to Present',
        'type': 'Shapefile',
        'description':'All collisions in the City of Los Angeles to date',
        'tags': 'Vision Zero'}
#     shp_item = gis.content.add(item_properties=shp_properties,
#                           data = shp_path)
    shp_item = Item(gis, itemid='973de14c027d4752b8bc264b4b66136a')
    shp_item.update(item_properties=shp_properties,
                   data = shp_path)
#     shp_item.publish(overwrite=True)

##### Create a dictionary using the first row of the sheet as the keys and row values as value pairs
def CollisionDict(rows, first_row):
    
    lapd_data = []
    for row in rows:
        record = {}
        # Create a dictionary based on the attributes in the first row
        for key, cell in zip(first_row, row):

            # For MO Codes, we want to parse out important info
            if key == 'MOCODES':
                clean_codes = CleanMOCodes(cell.value)
                for attr, val in clean_codes.items():
                    record[attr] = val
                record[key] = cell.value
                
            else:
                record[key] = cell.value

        lapd_data.append(record)
        #print(record)
    
    return lapd_data
    
##### Sort and Parse MO Codes
def CleanMOCodes(list_of_codes):
    
    # Dictionary keys that we will add later
    dict = {}
    flds = ['collision_severity', 'ped_inv', 'bike_inv', 'mc_inv','hit_and_run', 'intersection']
    for fld in flds:
        dict[fld] = ''
    
    # Convert str to int
    mo_codes_str = list_of_codes.split()
    mo_codes = sorted(list(map(int, mo_codes_str)), key=int)
    
    # Codes for ped/bike/mc/hit&run/collision severity
    ped_inv_codes = [3003]
    bike_inv_codes = [3008, 3016, 3017, 3018]
    mc_inv_codes = [3009, 3013, 3014, 3015]
    hit_and_run_codes = [3029, 3030]
    
    # Check collision severity
    if 3027 in mo_codes:
        dict['collision_severity'] = 1
    elif 3024 in mo_codes:
        dict['collision_severity'] = 2
    elif 3025 in mo_codes:
        dict['collision_severity'] = 3
    elif 3026 in mo_codes:
        dict['collision_severity'] = 4
    elif 3028 in mo_codes:
        dict['collision_severity'] = 0
    
    # Check for bike/ped/mc involved & hit and runs
    #if mo_codes.intersection(ped_inv_codes) is not False:
    if len([i for i in mo_codes if i in ped_inv_codes]) > 0:
        dict['ped_inv'] = 'Y'
    #if mo_codes.intersection(bike_inv_codes) is not False:
    if len([i for i in mo_codes if i in bike_inv_codes]) > 0:
        dict['bike_inv'] = 'Y'
    #if mo_codes.intersection(mc_inv_codes) is not False:
    if len([i for i in mo_codes if i in mc_inv_codes]) > 0:
        dict['mc_inv'] = 'Y'
    #if mo_codes.intersection(hit_and_run_codes) is not False:
    if len([i for i in mo_codes if i in hit_and_run_codes]) > 0:
        dict['hit_and_run'] = 'Y'
        
    # Check for intersection-related collisions
    if 3036 in mo_codes:
        dict['intersection'] = 'Y'
    elif 3037 in mo_codes:
        dict['intersection'] = 'N'
    
    # Return dictionary
    return dict

##### Zip an entire directory
def makeArchive(fileList, archive, root):
    """
    'fileList' is a list of file names - full path each name
    'archive' is the file name for the archive with a full path
    """
    a = zipfile.ZipFile(archive, 'w', zipfile.ZIP_DEFLATED)

    for f in fileList:
        print("archiving file %s" % (f))
        a.write(f, os.path.relpath(f, root))
    a.close()

if __name__ == '__main__':
    main()



171204180 POINT(6475264.50013794 1812572.2940953523)
170104026 POINT(6486875.499881774 1837775.7177697718)
172104003 POINT(6382426.998805478 1900638.9965705425)
171404009 POINT(6441804.347242102 1832275.8501605988)
171204176 POINT(6473303.001764402 1823859.3745395541)
171904005 POINT(6431000.995161235 1912928.7511904687)
170604001 POINT(6467065.750131279 1856896.1641088575)
170204026 POINT(6476619.001037896 1840797.000441134)
171212366 POINT(6471991.501434639 1820051.1232977957)
171204175 POINT(6468022.500186309 1820403.7512014657)
170104014 POINT(6483774.004772484 1841506.6292712092)
170104023 POINT(6484088.996268481 1837880.8740555495)
171204142 POINT(6471991.501434639 1820051.1232977957)
171404047 POINT(6430393.81264852 1816133.213154599)
172004050 POINT(6470531.498449147 1843521.9953192174)
170704019 POINT(6457541.799999997 1849026.799999997)
170704002 POINT(6457941.501647562 1837058.873402968)
171504042 POINT(6436549.4984999 1893223.1209823787)
170304130 POINT(6457658.499588892 18

170304879 POINT(6461409.998319477 1822554.3778057098)
170904124 POINT(6420642.505051479 1880061.7532763034)
171504119 POINT(6449015.503751561 1883778.1277332902)
170404078 POINT(6500588.495093569 1838486.8790771365)
170604368 POINT(6462440.999866366 1856890.123766601)
170804086 POINT(6427502.750189185 1864460.592221439)
170704116 POINT(6449578.698491275 1851018.8776351064)
170104122 POINT(6487647.499650151 1837646.4955873042)
171004064 POINT(6401112.996538401 1886453.873037964)
170604107 POINT(6468060.499782234 1855564.626089543)
170304133 POINT(6472011.000083283 1830466.644789517)
171004065 POINT(6399547.004263565 1887406.0010549575)
170804091 POINT(6424177.996668145 1839473.9984939694)
170704068 POINT(6461370.503975898 1837059.503322959)
171104337 POINT(6474665.999038398 1860213.1217138022)
172104090 POINT(6375151.497280389 1897438.1283003837)
171104084 POINT(6468047.501448646 1860879.3720217198)
171804090 POINT(6475942.4984539 1797391.6253097951)
170604101 POINT(6457486.998619556 18

170104287 POINT(6481021.5 1836458.700000003)
171404138 POINT(6435485.002269283 1819574.9697225243)
171104151 POINT(6480083.003296226 1852332.000008881)
170704111 POINT(6456133.003624141 1844017.6235930473)
171904180 POINT(6431731.004856065 1934547.626915887)
171304180 POINT(6483929.500524476 1824241.5007437915)
170904265 POINT(6432588.319414362 1893899.9339331836)
170104221 POINT(6481927.295867443 1840014.938356936)
170604152 POINT(6467065.750131279 1856896.1641088575)
171604178 POINT(6450699.498415068 1903178.2539592087)
171504196 POINT(6445423.499452651 1898471.3745934665)
170904206 POINT(6421341.502997309 1889643.4949395508)
171704162 POINT(6399399.119716778 1904649.9997033477)
170104258 POINT(6486481.250014111 1843400.7734487653)
171404140 POINT(6435485.002269283 1819574.9697225243)
171904174 POINT(6434433.501419559 1937418.62642321)
171704168 POINT(6409944.996918976 1921758.502559796)
171704191 POINT(6385915.000031561 1907395.5007740408)
170904229 POINT(6428626.497613639 1895896.6

171204680 POINT(6471329.250134438 1819646.63952668)
170804185 POINT(6427502.750189185 1864460.592221439)
171104198 POINT(6489757.001336649 1864746.252411291)
170504259 POINT(6478703.495884642 1742734.1257011294)
171004214 POINT(6401108.002781972 1885525.628831461)
171004215 POINT(6398383.539398864 1894020.2804772705)
171104235 POINT(6501727.500033364 1861472.0171618462)
171704334 POINT(6394351.999889806 1908545.746176958)
171204572 POINT(6475955.500396401 1811865.1242498755)
170804178 POINT(6427204.179264024 1836500.0671058595)
170604371 POINT(6461825.299999997 1858145.900000006)
171404274 POINT(6448944.997658983 1814341.4995464683)
170204205 POINT(6480510.949290693 1846352.7048808485)
171904234 POINT(6423390.000540733 1905751.1247977167)
171304329 POINT(6486130.002685145 1834438.4974101335)
171304323 POINT(6477308.004581153 1818566.8792190403)
171004185 POINT(6402192.000942066 1888377.126737386)
171504317 POINT(6437830.49987486 1889916.9123716801)
171404188 POINT(6421503.996438727 181

171004272 POINT(6402209.998609483 1893353.0016281307)
172004240 POINT(6475577.49721539 1842828.497690469)
171504431 POINT(6441733.9999419 1882629.0027349591)
171504371 POINT(6441733.9999419 1882629.0027349591)
171004267 POINT(6413242.626150444 1896738.01060611)
170404270 POINT(6495181.501314402 1832667.497999385)
170604285 POINT(6457490.998283476 1857556.4957934618)
170404269 POINT(6500193.750148103 1851605.1044810265)
172005154 POINT(6474079.299999997 1836324.5)
171904312 POINT(6429088.502923235 1907445.8740328848)
171304362 POINT(6481269.996329069 1813298.2519610524)
171704354 POINT(6411954.499133557 1931276.2538654655)
170104414 POINT(6487821.496021315 1839930.8738925457)
171104285 POINT(6480914.499992773 1852086.054306522)
171704333 POINT(6411954.499133557 1931276.2538654655)
170204358 POINT(6481788.799999997 1844254.400000006)
171704309 POINT(6409984.500776976 1925047.3781998754)
171704307 POINT(6378527.996760979 1913903.1262102127)
171304361 POINT(6479167.499221727 1832129.878667

...

171412707 POINT(6442187.5010033995 1802823.3772415519)
171811924 POINT(6476534.501863062 1805005.7547665536)
170612241 POINT(6462774.501495972 1857547.2510612905)
170510382 POINT(6471994.21205911 1772543.1061092764)
170810356 POINT(6425988.499280736 1835154.7508988827)
172011105 POINT(6468052.496189311 1848421.503236711)
170612267 POINT(6460744.700000003 1856799.400000006)
170810351 POINT(6429368.004660815 1835854.75343588)
171110664 POINT(6495639.2500707805 1857028.2179785967)
172011040 POINT(6472008.750087783 1836526.9817501903)
172110863 POINT(6380967.496475145 1888155.1275094599)
170115001 POINT(6488277.7106600255 1844788.0744799376)
170612243 POINT(6464617.613295272 1858876.2634951025)
171110748 POINT(6485004.400000006 1859979.200000003)
170911196 POINT(6424671.99666439 1897232.8796951324)
170612262 POINT(6460167.502116814 1856906.1224222928)
171110693 POINT(6480978.500192866 1854561.5519801825)
171609631 POINT(6434891.502472058 1912490.3744425476)
171213920 POINT(6461382.49574981

171512220 POINT(6446841.799999997 1885310.200000003)
171512144 POINT(6451692.002991736 1893173.2516595423)
170210889 POINT(6479536.002093643 1839896.496336624)
171609731 POINT(6444505.599999994 1903856.900000006)
171312775 POINT(6479754.799999997 1825624.599999994)
170612291 POINT(6468044.498501897 1858678.628063634)
171110747 POINT(6483183.498407558 1857837.628433302)
171110725 POINT(6503035.003323898 1865698.3755070418)
171412807 POINT(6430982.496182486 1828862.3783130497)
171412808 POINT(6430982.496182486 1828862.3783130497)
170612766 POINT(6463663.496765226 1857863.8773965538)
170410244 POINT(6500772.502271473 1829287.8718846291)
171412837 POINT(6441490.001742229 1805663.7491854578)
170710210 POINT(6454158.500027195 1853031.7892916799)
170210883 POINT(6476501.996350646 1849193.999395132)
171811824 POINT(6477608.500043109 1806489.5008147657)
172011015 POINT(6474466.750093028 1839163.4236557633)
171412855 POINT(6445849.885082781 1805389.7494053543)
170612653 POINT(6469363.750128448 1

4326
written shapefile to disk.
archiving file Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp\collisions.cpg
archiving file Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp\collisions.dbf
archiving file Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp\collisions.prj
archiving file Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp\collisions.shp
archiving file Z:/GIS/DataLibrary/LAPD_CrimeCollision/Tables/Raw_Collisions_fromDianeWeber/PostGIS_Shapefile_Output/shp\collisions.shx


User cant overwrite this service, using this data, as this data is already referring to another service.


RuntimeError: User cant overwrite this service, using this data, as this data is already referring to another service.
(Error Code: 500)